# **LINEAR PROGRAMMING AND ZERO SUM GAMES**

In [ ]:
!pip install nashpy

**Re-formulation-of-the-linear-program**

In a zero game, given a row player payoff matrix \(A\) with \(m\) rows and \(n\) columns, the following linear program will give the max-min strategy and value:

$$\min\limits_{x \in \mathbb{R}^{(m + 1) \times 1}}cx$$

Subject to:

\begin{array}{r}
\begin{matrix}
{M_{\text{ub}}x} & {\leq b_{\text{ub}}} & & \\
{M_{\text{eq}}x} & {= b_{\text{eq}}} & & \\
x_{i} & {\geq 0} & & {\text{~for~}i \leq m} \\
\end{matrix} \\
\end{array}

Where the coefficients of the linear program are defined by:

\begin{array}{r}
\begin{matrix}
c & {= (\underset{m}{\underbrace{0,\ldots,0}}, - 1)} & & {c \in \{ 0,1\}^{1 \times (m + 1)}} \\
M_{\text{ub}} & {= \begin{pmatrix}
{( - A^{T})_{11}} & \ldots & {( - A^{T})_{1m}} & 1 \\
 \vdots & \ddots & \vdots & 1 \\
{( - A^{T})_{n1}} & \ldots & {( - A^{T})_{nm}} & 1 \\
\end{pmatrix}} & & {M \in \mathbb{R}^{n \times (m + 1)}} \\
b_{\text{ub}} & {= (\underset{n}{\underbrace{0,\ldots,0}})^{T}} & & {b_{\text{ub}} \in \{ 0\}^{n \times 1}} \\
M_{\text{eq}} & {= (\underset{m}{\underbrace{1,\ldots,1}},0)} & & {M_{\text{eq}} \in \{ 0,1\}^{1 \times (m + 1)}} \\
b_{\text{eq}} & {= 1} & & \\
\end{matrix} \\
\end{array}



In [ ]:
import numpy as np
from scipy.optimize import linprog
import nashpy as nash
np.set_printoptions(precision = 2)
def find_max_min_strategies(payoff_matrix):
    m, n = payoff_matrix.shape

    ### Max-min strategy for the row player
    c_row = np.concatenate([np.zeros(m), [-1]])
    M_ub_row = np.concatenate([np.transpose(-payoff_matrix), np.ones((n, 1))], axis=1)
    b_ub_row = np.zeros(n)
    M_eq_row = np.concatenate([np.ones(m), [0]])
    b_eq_row = 1
    bounds_row = [(0, None)] * m + [(None, None)]
    result_row = linprog(c_row, A_ub=M_ub_row, b_ub=b_ub_row, A_eq=[M_eq_row], b_eq=[b_eq_row], bounds=bounds_row, method='highs')
    max_min_strategy_row = result_row.x[:-1]
    max_min_value_row = -result_row.fun

    ### Min-max strategy for the column player
    c_col = np.concatenate([np.zeros(n), [1]])
    M_ub_col = np.concatenate([payoff_matrix, -np.ones((m, 1))], axis=1)
    b_ub_col = np.zeros(m)
    M_eq_col = np.concatenate([np.ones(n), [0]])
    b_eq_col = 1
    bounds_col = [(0, None)] * n + [(None, None)]
    result_col = linprog(c_col, A_ub=M_ub_col, b_ub=b_ub_col, A_eq=[M_eq_col], b_eq=[b_eq_col], bounds=bounds_col, method='highs')
    min_max_strategy_col = result_col.x[:-1]
    min_max_value_col = result_col.fun

    return max_min_strategy_row, max_min_value_row, min_max_strategy_col, min_max_value_col

matching_pennies = np.array([
    [1,-1],
    [-1,1]
])
max_min_strategy_row, max_min_value_row, min_max_strategy_col, min_max_value_col = find_max_min_strategies(matching_pennies)

print("Max-Min Strategy (Row Player):", max_min_strategy_row)
print("Min-Max Strategy (Column Player):", min_max_strategy_col)


Max-Min Strategy (Row Player): [0.5 0.5]
Min-Max Strategy (Column Player): [0.5 0.5]


In [ ]:

game = nash.Game(matching_pennies)
game.linear_program()

(array([0.5, 0.5]), array([0.5, 0.5]))

In [ ]:
def generate_payoff_matrix(n):
    ### Generate random integers for the payoff matrix
    payoff_matrix = np.random.randint(-10, 10, size=(n, n))

    return payoff_matrix
n = 5
random_payoff_matrix = generate_payoff_matrix(n)

print("Random Payoff Matrix:")
print(random_payoff_matrix)


Random Payoff Matrix:
[[ -8   7   3  -3  -1]
 [  2  -6  -6   3  -6]
 [ -8  -9 -10   4   1]
 [ -4  -8   9   5   0]
 [  5   1  -7  -1  -9]]


In [ ]:


max_min_strategy_row, max_min_value_row, min_max_strategy_col, min_max_value_col = find_max_min_strategies(random_payoff_matrix)

print("Max-Min Strategy (Row Player):", max_min_strategy_row)
print("Min-Max Strategy (Column Player):", min_max_strategy_col)

Max-Min Strategy (Row Player): [0.21 0.   0.   0.53 0.26]
Min-Max Strategy (Column Player): [0.37 0.13 0.   0.   0.5 ]


In [ ]:
game = nash.Game(random_payoff_matrix)
game.linear_program()

(array([0.21, 0.  , 0.  , 0.53, 0.26]), array([0.37, 0.13, 0.  , 0.  , 0.5 ]))

# **NORMAL FORM GAMES**

# **DOUBLE ORACLE ALGORITHM**

In [ ]:
def solve_reduced_matrix(payoff_matrix, R, C):
  ### Solve the reduced matrix using linear programming and return the optmal policy for this reduced matrix
  q, p = np.zeros(len(R)), np.zeros(len(C))
  reduced_matrix = payoff_matrix[R == 1, :][:, C == 1]
  game = nash.Game(reduced_matrix)
  q_, p_ = game.linear_program()
  q[R==1], p[C==1] = q_, p_
  return q, p


def get_pure_strategy(matrix, q, p):
  ### Get the pure strategies
  r = np.argmax(np.dot(matrix, p))
  c = np.argmax(np.dot(q, - matrix))
  return r, c

def double_oracle_algorithm(matrix):
  ### The double oracle algorithm
  R, C = np.zeros(matrix.shape[0]), np.zeros(matrix.shape[1])
  R[0], C[0] = 1, 1
  history_q , history_p = [], []
  while True:
    q, p = solve_reduced_matrix(matrix, R, C)
    history_p.append(p)
    history_q.append(q)
    r, c = get_pure_strategy(matrix, q, p)
    if R[r] == 1 and C[c] == 1:
      return history_q, history_p
    else:
      R[r], C[c] = 1, 1

### Generate random normal form games
random_payoff_matrix = generate_payoff_matrix(100)

%timeit history_q, history_p = double_oracle_algorithm(random_payoff_matrix)

print("Players 1 optimal policy: ", history_q[-1])
print("Players 2 optimal policy: ", history_p[-1])


1.08 s ± 283 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Players 1 optimal policy:  [0.01 0.01 0.02 0.   0.01 0.03 0.   0.   0.   0.   0.01 0.   0.03 0.02
 0.   0.   0.   0.   0.01 0.   0.   0.02 0.   0.01 0.   0.   0.   0.
 0.   0.03 0.03 0.   0.01 0.   0.02 0.   0.03 0.01 0.01 0.   0.01 0.
 0.03 0.   0.   0.03 0.   0.   0.   0.02 0.01 0.05 0.   0.01 0.   0.
 0.01 0.01 0.01 0.   0.   0.01 0.07 0.   0.01 0.   0.03 0.   0.03 0.
 0.   0.01 0.   0.   0.   0.   0.   0.02 0.   0.05 0.   0.   0.03 0.
 0.02 0.02 0.03 0.02 0.   0.04 0.01 0.03 0.   0.02 0.   0.   0.   0.
 0.   0.03]
Players 2 optimal policy:  [0.   0.   0.   0.01 0.01 0.02 0.02 0.   0.04 0.   0.   0.   0.03 0.
 0.05 0.01 0.   0.05 0.01 0.   0.01 0.   0.02 0.   0.   0.02 0.   0.01
 0.03 0.   0.02 0.04 0.01 0.04 0.   0.   0.   0.03 0.01 0.   0.02 0.
 0.03 0.01 0.   0.   0.04 0.03 0.   0.   0.02 0.02 0.01 0.   0.01 0.
 0.02 0.02 0.   0.   0.   0.   0.02 0.02 0.   0.   0.06 0.01 0.   0.
 0.02 0.01 0.01 0.03 0.01 0.   0.  

In [ ]:
### Jax implementation

import jax
import jax.numpy as jnp

%timeit
def generate_payoff_matrix(size):
    return jax.random.uniform(jax.random.PRNGKey(0), shape=(size, size))

def solve_reduced_matrix(payoff_matrix, R, C):
    q, p = jnp.zeros(len(R)), jnp.zeros(len(C))

    reduced_matrix = payoff_matrix[R == 1, :][:, C == 1]
    game = nash.Game(reduced_matrix)
    q_, p_ = game.linear_program()
    q = q.at[R == 1].set(q_)
    p = p.at[C == 1].set(p_)
    return q, p

def get_pure_strategy(matrix, q, p):
    r = jnp.argmax(jnp.dot(matrix, p))
    c = jnp.argmax(jnp.dot(q, -matrix))
    return r, c

def double_oracle_algorithm(matrix):
    R, C = jnp.zeros(matrix.shape[0]), jnp.zeros(matrix.shape[1])
    R = R.at[0].set(1)
    C = C.at[0].set(1)
    history_q, history_p = [], []

    while True:
        q, p = solve_reduced_matrix(matrix, R, C)
        history_p.append(p)
        history_q.append(q)
        r, c = get_pure_strategy(matrix, q, p)

        if R[r] == 1 and C[c] == 1:
            return history_q, history_p
        else:
            R = R.at[r].set(1)
            C = C.at[c].set(1)

random_payoff_matrix = generate_payoff_matrix(100)

%timeit history_q, history_p = double_oracle_algorithm(random_payoff_matrix)
print("Players 1 optimal policy: ", history_q[-1])
print("Players 2 optimal policy: ", history_p[-1])

1.73 s ± 116 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Players 1 optimal policy:  [0.01 0.01 0.02 0.   0.01 0.03 0.   0.   0.   0.   0.01 0.   0.03 0.02
 0.   0.   0.   0.   0.01 0.   0.   0.02 0.   0.01 0.   0.   0.   0.
 0.   0.03 0.03 0.   0.01 0.   0.02 0.   0.03 0.01 0.01 0.   0.01 0.
 0.03 0.   0.   0.03 0.   0.   0.   0.02 0.01 0.05 0.   0.01 0.   0.
 0.01 0.01 0.01 0.   0.   0.01 0.07 0.   0.01 0.   0.03 0.   0.03 0.
 0.   0.01 0.   0.   0.   0.   0.   0.02 0.   0.05 0.   0.   0.03 0.
 0.02 0.02 0.03 0.02 0.   0.04 0.01 0.03 0.   0.02 0.   0.   0.   0.
 0.   0.03]
Players 2 optimal policy:  [0.   0.   0.   0.01 0.01 0.02 0.02 0.   0.04 0.   0.   0.   0.03 0.
 0.05 0.01 0.   0.05 0.01 0.   0.01 0.   0.02 0.   0.   0.02 0.   0.01
 0.03 0.   0.02 0.04 0.01 0.04 0.   0.   0.   0.03 0.01 0.   0.02 0.
 0.03 0.01 0.   0.   0.04 0.03 0.   0.   0.02 0.02 0.01 0.   0.01 0.
 0.02 0.02 0.   0.   0.   0.   0.02 0.02 0.   0.   0.06 0.01 0.   0.
 0.02 0.01 0.01 0.03 0.01 0.   0.  